In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
import os
import json
%matplotlib inline
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict


In [2]:
#pip install tqdm
import tqdm

In [3]:
def loop_slices(path, num_slices=20):
  cnt = 0
  mpd_playlists = []
  filenames = os.listdir(path)
  for fname in sorted(filenames):
    print(fname)
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
      cnt += 1
      fullpath = os.sep.join((path, fname))
      f = open(fullpath)
      js = f.read()
      f.close()
      current_slice = json.loads(js)
      # Create a list of all playlists
      for playlist in current_slice['playlists']:
        mpd_playlists.append(playlist)
      if cnt == num_slices:
        break
  return mpd_playlists
# Path where the json files are extracted
path = '/Users/Ethan Vaz Falcao/Documents/Spotify/spotify_million_playlist_dataset/data'
playlists = loop_slices(path, num_slices=2)

mpd.slice.0-999.json
mpd.slice.1000-1999.json


In [9]:
import os
import time
import random
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm

# Spotify credentials
os.environ["SPOTIPY_CLIENT_ID"] = "520cb282dc3541b19f6c8d7d182770ab"
os.environ["SPOTIPY_CLIENT_SECRET"] = "1bd9d4b4bfcf49c590b1cd96f4f7cf69"
os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:8080"

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

cols_to_keep = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                'duration_ms', 'time_signature']
dfs = []

# Assuming you have defined the playlists variable as a list of playlists
for playlist in tqdm(playlists):
    audio_feats = []
    for track in playlist['tracks']:
        track_uri = track['track_uri'].split(":")[2]
        retries = 3  # Maximum number of retries
        for attempt in range(retries):
            try:
                feature = sp.audio_features(track_uri)
                if feature:
                    audio_feats.append(feature[0])
                break  # Exit the retry loop if successful
            except spotipy.SpotifyException as e:
                if e.http_status == 429:
                    wait_time = 2 ** attempt + random.random()  # Exponential backoff formula
                    time.sleep(wait_time)
                    continue  # Retry the request
                else:
                    raise  # Raise other SpotifyExceptions
    avg_feats = pd.DataFrame(audio_feats)[cols_to_keep].mean()
    avg_feats['name'] = playlist['name']
    avg_feats['pid'] = playlist['pid']
    dfs.append(avg_feats.T)


  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
Max Retries reached
M

KeyError: "None of [Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',\n       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',\n       'duration_ms', 'time_signature'],\n      dtype='object')] are in the [columns]"